In [2]:
import os
import pandas as pd
import requests
from tqdm import tqdm_notebook
import glob
import json

In [27]:
# Считываем данные из таблицы (CSV или Excel, в зависимости от формата)
file_path = '../../raw_fields_with_transliteration.pq'  # укажите правильный путь к файлу
df = pd.read_parquet(file_path)  # если у вас Excel, используйте pd.read_excel(file_path)

In [28]:
df.sort_values('index orig',inplace=True)

In [29]:
df

,index orig,link,description,title shazam,subtitle shazam,transcription whisper v3,image captioning,title shazam transliterated,subtitle shazam transliterated
37499,0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m...",Resurrection,Tapiuanthegod,Thank you.,a soccer game with players on the field,Воскресение,Tapiuanthefod
37500,1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...,,,Сейчас я тебе покажу секретную команду в Робл...,a picture of a man in a dark room with the wo...,"Я не знаю, что делать.","Я не знаю, что делать."
37501,2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass",Substance,Greedo 03,I wanna spend some time with her Cause I can'...,a man and woman in blue paint are kissing,Вещество,Жадность 03
37502,3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,,,,"Что он о тебе думает прямо сейчас? Слушай, ду...",a woman is sitting at a table with cards and ...,"Я не знаю, что делать.","Я не знаю, что делать."
37503,4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,,,,"Я убежден, что чем беднее человек, тем сложне...",a man in a purple suit is sitting at a desk,"Я не знаю, что делать.","Я не знаю, что делать."
...,...,...,...,...,...,...,...,...,...
16994,39995,https://cdn-st.rutubelist.ru/media/23/be/acad5...,#спорт #наспорте #хоккей #игра #хкгуу #гол,3500 Chainz 2 Feat Future,Scott Travis,Only true niggas I know Only true niggas I know,a hockey game is being played on an ice rink,3 500 Chainz 2 Feat Future,Скотт Трэвис
16995,39996,https://cdn-st.rutubelist.ru/media/57/f3/ed28e...,"#boobs , #красивыедевушки , #ass",Life Married,Luciani Gina,Okay!,a person is putting something in a bowl on a ...,Жизнь замужем,Лусиани Джина
16996,39997,https://cdn-st.rutubelist.ru/media/90/6a/fa7be...,#красивыедевушки #грудь #boobs #попа #ass #big...,Eyes Close,Dvrst,자막 제공 배달의민족,a woman with blue hair and a blue face,Закрыть глаза,Dvrst
16997,39998,https://cdn-st.rutubelist.ru/media/58/0d/8d9a3...,Game 7😤 TIMBERWOLVES vs The NUGGETS…Ant-Man vs...,,,Teksting av Nicolai Winther,two men in blue uniforms standing on a basket...,"Я не знаю, что делать.","Я не знаю, что делать."


In [30]:
df.reset_index(drop=True,inplace=True)

## проба [saiga_llama3_8b](https://huggingface.co/IlyaGusev/saiga_llama3_8b) базово

это для запуска на cpu - [IlyaGusev/saiga_llama3_8b_gguf](https://huggingface.co/IlyaGusev/saiga_llama3_8b_ggufv)

In [56]:
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [23]:
path_to_models :str = '/home/jovyan/models'

In [24]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=path_to_models
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=path_to_models)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME, cache_dir=path_to_models)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
def generate_llm(
    text: str, model: AutoModelForCausalLM, tokenizer: AutoTokenizer, config: dict
) -> str:
    """
    Принимает на вход один сэмпл текста, который необходимо аугментировать с помощью LLM

    Args:
        text (str): Текст, который нужно аугментировать
        model (AutoModelForCausalLM): LLM модель
        tokenizer (AutoTokenizer): Токенизатор
        config (dict): Конфиг, где определены основные параметры генерации, а также системный промпт и юзер промпт

    Returns:
        str: Аугментированное предложение
    """
    input_messages = [
        {
            "role": "system",
            "content": config["prompts"]["system_prompt"],
        },
        {
            "role": "user",
            "content": config["prompts"]["user_prompt"].format(message=text),
        },
    ]
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    ]

    input_ids = tokenizer.apply_chat_template(
        input_messages, add_generation_prompt=True, return_tensors="pt", tokenize=True
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=config["generation_parameters"]["max_new_tokens"],
        eos_token_id=terminators,
        do_sample=config["generation_parameters"]["do_sample"],
        temperature=config["generation_parameters"]["temperature"],
        top_p=config["generation_parameters"]["top_p"],
        top_k=config["generation_parameters"]["top_k"],
        no_repeat_ngram_size=config["generation_parameters"]["no_repeat_ngram_size"],
    )

    response = outputs[0][input_ids.shape[-1] :]
    output_text = tokenizer.decode(response, skip_special_tokens=True)

    return output_text

In [111]:
DEFAULT_SYSTEM_PROMPT = """|
Сгенерируй топ-5 качественных коротких поисковых запроса, состоящих из одного или двух слов, с помощью которых можно легко найти короткий видеоролик длительностью от 15 и до 60 секунд.
Опирайся на транскрибацию аудио и описание фрейма видео.

На вход подаются следующие сущности, которые отделены между друг другом '||':
  - transcription: транскрибация аудио на любом языке 
  - image_caption: описание фрейма видео на английском

На выходе должно быть только JSON с ключом 'queries', значением которого является список из пяти наиболее подходящих коротких поисковых запросов для видео. Запросы должны быть короткими и точными, не превышать два слова.

Пример 1 входных данных :
transcription: Привет, это мой новый влог о путешествии в Париж!
||image_caption: a view of the Eiffel Tower in Paris

Пример 1 выходного JSON :
{
   "queries": [
      "влог о путешествии",
      "путешествия",
      "Париж",
      "Eiffel Tower",
      "Paris travel",
   ]
}

Пример 2 входных данных :
transcription: В этом видео я расскажу, как приготовить вкуснейший шоколадный торт всего за 30 минут!
||image_caption: a close-up of a chocolate cake with strawberries on top

Пример 2 выходного JSON :
{
   "queries": [
      "шоколадный торт",
      "торт рецепт",
      "приготовление торта",
      "cake",
      "шоколад"
   ]
}

Пример 3 входных данных :
transcription: В этом ролике мы рассмотрим основные функции последней модели iPhone и сравним ее с предыдущей версией.
||image_caption: a new iPhone model with its box

Пример 3 выходного JSON :
{
   "queries": [
      "новый iPhone",
      "iPhone обзор",
      "iPhone функции",
      "iPhone сравнение",
      "смартфон"
   ]
}
"""

In [112]:
max_new_tokens = 512
temperature = 0.2

In [121]:
config = {
    "prompts": {
        "system_prompt": DEFAULT_SYSTEM_PROMPT,
        "user_prompt": """|
                           {message}
                           """,
    },
    "generation_parameters": {
        "max_new_tokens": max_new_tokens,
        "do_sample": True,
        "temperature": temperature,
        "top_k": 30,
        "top_p": 0.9,
        "no_repeat_ngram_size":None
    },
}

In [122]:
delimiter: str = "||"

In [123]:
indexes = range(0, 5)

In [124]:
answers: list[int] = []

all_outputs :list[str] = []

start_time = time.time()
for index in tqdm_notebook(indexes):
    row = df.iloc[index]
    text: str = f"transcription:{row['transcription whisper v3']}{delimiter}image_caption:{row['image captioning']}"

    output_text = generate_llm(text, model, tokenizer, config)

    try:
        result = json.loads(output_text.strip('```json\n```'))
        answer = result["queries"]
    except Exception as e:
        print(e)
        answer = ""
    answers.append(answer)
    all_outputs.append(output_text)
exec_time = time.time()-start_time
mean_time = (exec_time)/len(all_outputs)

/home/jovyan/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [130]:
print(f"exec_time: {exec_time:.2f} sec.")
print(f"mean_time: {mean_time:.2f} sec.")

exec_time: 39.43 sec.
mean_time: 7.89 sec.


In [126]:
for index in indexes:
    row = df.iloc[index]
    text: str = f"transcription:{row['transcription whisper v3']}\n{delimiter}image_caption:{row['image captioning']}"
    print(f"index:{index}")
    print(20*'-')
    print(f"text for llm:{text}")
    print(20*'-')
    print(f"output_text:{all_outputs[index]}")
    print(40*'=')
    print()

index:0
--------------------
text for llm:transcription: Thank you.
||image_caption: a soccer game with players on the field
--------------------
output_text:{
   "queries": [
      "сoccer",
      "soccer game",
      "футбол игра",
      "игра футбол",
      "спорт"
   ]
}

index:1
--------------------
text for llm:transcription: Сейчас я тебе покажу секретную команду в Роблоксе. Чтобы её активировать поставь лайк и подпишись, а также введи в чат команду ILOVEYOU. Когда вы её введёте у вас на экране появится вот такой клоун. Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблос девелопер сервис. Чтобы убрать скример нужно всего
||image_caption: a picture of a man in a dark room with the words bot takon
--------------------
output_text:{
   "queries": [
      "Роблокс секретная команда",
      "ILOVEYOU Roblox",
      "скриммер Roblox",
      "Roblox developer service",
      "клоун Roblox"
   ]
}

index:2
--------------------
text for llm:transcrip

**Итог:**

Модель `IlyaGusev/saiga_llama3_8b` ( при подборе промпта, параметров генерации модели и few-shot ) генерирует полезные хэштеги для пользователей и поиска, которые можно в дальнейшем использовать. 

Время работы генерации около 8 секунд на GPU, что дольше чем вся текущая индексация видео. Поэтому данное решение можно использовать как генерацию хэштегов видео для пользователей, которые не писали ничего в поле `описание видео` и предлагать им отредактировать автосгенерированные хэштеги.

Кроме того, потребление модели `IlyaGusev/saiga_llama3_8b` загруженной в 8 битах (load_in_8bit=True) от 10 до 13 Гб  GPU поэтому для ее использования нужна отдельный сервер GPU.